In [10]:
import requests
import pandas as pd
import time

def get_top_assets():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print("Error fetching top assets:", response.json())
        return []

    data = response.json()

    asset_list = [coin["id"] for coin in data if coin["id"] not in stablecoins]
    
    return asset_list

# Function to get historical data for an asset
def get_historical_data(asset_id, days=30):
    url = f"https://api.coingecko.com/api/v3/coins/{asset_id}/market_chart"
    params = {
        "vs_currency": "usd",
        "days": days,
        "interval": "daily"
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f"Error fetching {asset_id}: {response.json()}")
        return None

    data = response.json()
    
    # Check if "prices" key exists
    if "prices" not in data:
        print(f"Skipping {asset_id}: No historical price data available")
        return None

    # Convert to DataFrame
    prices = data["prices"]  # List of [timestamp, price]
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    
    # Convert timestamp to date
    df["date"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("date", inplace=True)
    
    # Calculate percentage price change
    df["price_change_pct"] = df["price"].pct_change() * 100

    return df

# Get top 100 assets
top_assets = get_top_assets()

# Collect data for each asset
all_data = {}
for asset in top_assets:
    try:
        print(f"Fetching data for {asset}...")
        df = get_historical_data(asset)
        if df is not None:
            all_data[asset] = df
        time.sleep(30)  # Increase delay to 10 seconds to avoid rate limits
    except Exception as e:
        print(f"Unexpected error fetching {asset}: {e}")

# Convert to a single DataFrame
df_list = [df.assign(asset=asset) for asset, df in all_data.items()]
final_df = pd.concat(df_list)

# Save to CSV
final_df.to_csv("top_100_crypto_historical_fixed.csv")

print("Data collection complete!")


Fetching data for bitcoin...
Fetching data for ethereum...
Fetching data for ripple...
Fetching data for binancecoin...
Fetching data for solana...
Fetching data for cardano...
Fetching data for dogecoin...
Fetching data for tron...
Fetching data for staked-ether...
Fetching data for wrapped-bitcoin...
Fetching data for pi-network...
Fetching data for chainlink...
Fetching data for leo-token...
Fetching data for the-open-network...
Fetching data for stellar...
Fetching data for wrapped-steth...
Fetching data for usds...
Fetching data for hedera-hashgraph...
Fetching data for avalanche-2...
Fetching data for shiba-inu...
Fetching data for sui...
Fetching data for litecoin...
Fetching data for bitcoin-cash...
Fetching data for polkadot...
Fetching data for mantra-dao...
Fetching data for ethena-usde...
Fetching data for weth...
Fetching data for bitget-token...
Fetching data for binance-bridged-usdt-bnb-smart-chain...
Fetching data for hyperliquid...
Fetching data for whitebit...
Fetchin